# mobile-base-sdk example notebook

This notebook present aims at helping you get your hands on Reachy's mobile base's sdk python api.

You will learn how to connect to Reachy's mobile base, access basic information, send movement commands
and check advanced features.

For more information, check [Reachy's online documentation](https://docs.pollen-robotics.com/).

## Connect to the mobile base

With just two lines of code you can connect to Reachy's mobile base. Just like with [ReachySDK](https://docs.pollen-robotics.com/sdk/getting-started/introduction/), you can connect to the mobile base from any computer as long it is
on the same network as Reachy.

In [1]:
from mobile_base_sdk import MobileBaseSDK

The communication with the mobile base using its SDK goes through the instaciation of the MobileBaseSDK object.
MobileBaseSDK takes only one argument at initialisation, Reachy's IP address on the network. If you need help to find Reachy's IP address, check out the [Finding Reachy's IP section](https://docs.pollen-robotics.com/help/system/find-my-ip/) from the online documentation. If you're working directly on Reachy, you can just put *'localhost'* as IP address, like the example below.

In [2]:
mobile_base = MobileBaseSDK('localhost') # Replace with your Reachy's IP address

## At any time you can call the emergency stop if you have a problem with the mobile base

**NOTE :** calling the *mobile_base.emergency_shutdown()* method will kill the mobile base's hal and sdk server.

Thus, if you need to keep using the mobile base, you will need to use the *restart_hal* method.

In [4]:
mobile_base.emergency_shutdown()

Emergency shutdown executed.
No command on the mobile base will work until you restarted its hal and sdk server.


In [6]:
mobile_base.restart_hal()

## Utility commands

Once the connection with the mobile base established, you have access to information such as the mobile base's model version, battery voltage, drive mode and control mode.

In [7]:
mobile_base

<MobileBase host="localhost" - version=1.0 - battery_voltage=
        28.1 - drive mode=cmd_vel - control mode=open_loop>

In [4]:
mobile_base.model_version

1.0

In [7]:
mobile_base.battery_voltage

28.0

## Control modes

The control mode dictates the low level control strategy used by the mobile bases's hal.

Two control modes are possible:
* ***open_loop***(default mode): in this mode, the wheels are compliant and the control is smoother,


* ***pid***: in this mode, the wheels are stiff and the control is more precise.

The control mode on the mobile base can be get with the *control_mode* attribute:

In [7]:
mobile_base.control_mode

'open_loop'

The *control_mode* can also be set:

In [8]:
mobile_base.control_mode = 'pid'

In [9]:
mobile_base.control_mode

'pid'

## Odometry

You can have access to the mobile base odometry with the *odometry* attribute. The odometry is initialised when the *reachy_mobile_base.service* or the mobile base hal and sdk is started OR whenever the *reset_odometry* method is called.

**NOTE:** x and y are in meters and theta is in degree.

In [10]:
mobile_base.odometry

{'x': 0.016, 'y': -0.008, 'theta': -4.342}

In [11]:
mobile_base.reset_odometry()

In [12]:
mobile_base.odometry

{'x': 0.0, 'y': 0.0, 'theta': 0.0}

## Mobility commands

Two methods are available to make send moving instructions to the mobile base:
* **set_speed**: send translation speed instructions along the x and y axis (in m/s) and/or rotation speed (in deg/s) instruction to the mobile base. Note that the speed instructions will be applied for a **duration of 200ms**,


* **goto**: send cartesian target point (x, y in meters) in the odometry frame and a desired orientation (theta in degree) for the mobile base.

### set speed (x_vel in m/s, y_vel in m/s, rot_vel in deg/s)

In [13]:
mobile_base.set_speed(x_vel=0, y_vel=0, rot_vel=60)

### go to (x in m, y in m, theta in deg)
Go to in the odometry frame. After an odometry reset the point (0,0) is the current position of the robot with, x is in front of the robot, y to the left and theta counter-clockwise.

In [4]:
mobile_base.reset_odometry()
mobile_base.goto(x=0, y=0, theta=90)

You can also have access to the distance between the mobile base and its last goto target with the *_distance_to_goto_goal* attribute. 

In [18]:
mobile_base._distance_to_goto_goal()

{'delta_x': 0.002, 'delta_y': -0.003, 'delta_theta': -2.672, 'distance': 0.004}

## Advanced features

Here we mention some features of the mobile base that we consider more advanced than the others presented in this notebook. More info on the mobile base documentation.

### Drive mode
The drive mode on the mobile base will impact the way the wheels behave and the way they will accept commands.

Six drive modes are available for the mobile base:
* **cmd_vel**: in this mode, speed instructions can be spammed to the wheels controllers. This mode is used for the *set_speed* method.
* **brake**: in this mode, the wheels will be as stiff as possible.
* **free_wheel**: in this mode, the wheels will be as compliant as possible.
* **goto**: this mode is used for the *goto* method.
* **emergency_stop**: in this mode, the wheels will stop receiving mobility commands. Switching to this mode will also stop the mobile base hal code. This is a safety mode. 
* **speed**: another mode to send speed instructions, but less frequently than with the cmd_vel mode. This mode is actually not used at this level (python SDK level), but is implemented at the ROS level, in case one might need it.

The drive mode is handled automagically when requesting a set_speed or a goto. There are some cases where changing them by hand could be useful, such as making the mobile as compliant as possible ("free_wheel" mode) or the opposite ("brake" mode). For more details on all the available modes, check: https://github.com/pollen-robotics/zuuu_hal

In [20]:
mobile_base.drive_mode

'go_to'

In [10]:
mobile_base.drive_mode = "free_wheel"

In [11]:
mobile_base.drive_mode = "brake"

### async goto

An async version of the goto method is alvailable. It can be useful in cases where for example you want to combine a movement of the mobile base with a movement of reachy.

In [19]:
await mobile_base.goto_async(x=0, y=0, theta=90)